In [1]:
import numpy as np
import mne
from pymatreader import read_mat


### Create info from .mat

In [2]:
info_path = '/storage/store/data/biomag_challenge/Biomag2022/biomag_hokuto/traininig/control/control_01_25/hokuto_control10.mat'
infos = read_mat(info_path)
infos = infos['D']
infos.keys()

dict_keys(['type', 'Nsamples', 'Fsample', 'timeOnset', 'trials', 'channels', 'data', 'fname', 'path', 'sensors', 'fiducials', 'transform', 'condlist', 'montage'])

In [3]:
infos['data']

{'fname': 'R:\\biomag\\training\\control\\hokuto_control10.dat',
 'dim': array([   186, 600000], dtype=int32),
 'dtype': 16,
 'be': 0,
 'offset': 0,
 'pos': array([1, 1], dtype=uint8),
 'scl_slope': array([], dtype=uint8),
 'scl_inter': array([], dtype=uint8),
 'permission': 'rw'}

In [4]:
# sensor_position = data['sensors']
# fiducials = data['fiducials']
ch_names = infos['channels']['label']
ch_types = infos['channels']['type']
ch_types = ['grad' if ch=='MEGGRAD' else 'misc' if ch=='Other' else 'ref_meg' for ch in ch_types ]
sfreq = infos['Fsample']

In [5]:
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
info

<Info | 7 non-empty values
 bads: []
 ch_names: AG001, AG002, AG003, AG004, AG005, AG006, AG007, AG008, AG009, ...
 chs: 160 Gradiometers, 14 misc, 12 Reference Magnetometers
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 186
 projs: []
 sfreq: 2000.0 Hz
>

### Load data from .dat

In [7]:
data_path = '/storage/store/data/biomag_challenge/Biomag2022/biomag_hokuto/traininig/control/control_01_25/hokuto_control10.dat'
data = np.fromfile(data_path, dtype="float32")  # Ca doit pas etre le bon dtype
data = np.reshape(data, (186, 600000))

In [8]:
raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=186, n_times=600000
    Range : 0 ... 599999 =      0.000 ...   300.000 secs
Ready.


In [9]:
%matplotlib qt

In [10]:
raw.plot()

Using qt as 2D backend.
Opening raw-browser...
Free RAM space could not be determined because"psutil" is not installed. Setting precompute to False.


KeyError: 'figure.raise_window'

Closing raw-browser...
Channels marked as bad:
none
Closing raw-browser...
Channels marked as bad:
none
Closing raw-browser...
Channels marked as bad:
none
